每天有18项观测值, 每天的每项观测值有24项. 总共$12\times20=240$天

随机取24天*24小时的数据作为测试集

现在取连续的9小时的18项观测值的三次多项式为一个$\vec x_i$, 其维度再加上一bias为$(9\times18)*3+1=487$. 对应的系数向量$\vec w_i$维度也为487.

一共有(240-24)*24-9=5175笔训练数据, 姑且先写三次多项式拟合吧.

24*24-9=567笔测试数据

第10个小时的pm2.5值$y_i=\sum_{i=1}^{487}\vec x_i\cdot\vec w$.

Loss function 为$L(\vec w)=\sum_{i=1}^{487}(y_i - \vec w\cdot\vec x_i)^2+\lambda\|\vec w\|^2$

In [1]:
import re
import numpy as np
import math
import random

train_data_path = '/data/zys/regression/train.csv'
f = open(train_data_path, 'r', encoding='big5')

data = []
for i in range(0, 18):
    data.append([])
i = 0
for line in f.readlines():
    j = 0
    for word in re.split(',|\n', line):
        if i>0 and j>2:
            if word=='':
                continue
            if word=='NR':
                data[(i-1)%18].append(0.0)
            else:
                data[(i-1)%18].append(float(word))
        j = j+1
    i = i+1

f.close

sta = random.randint(0, len(data[0])-1)

随机取24份连续24小时的数据

In [2]:
temp = []
for i in range(0, 18):
    temp.append([])
for i in range(0, 24):
    for k in range(0, 18):
        for j in range(0, 24):
            temp[k].append(data[k][sta+j])
        for j in range(0,24):
            data[k].pop(sta+j)

test_x = []
test_y = []            

for i in range(0, len(temp[0])-9 ):
    test_x.append([])
    for j in range( 0, 9):
        for k in range(0, 18):
            test_x[i].append(temp[k][i+j])
            test_x[i].append(temp[k][i+j]**2)
            test_x[i].append(temp[k][i+j]**3)
    test_x[i].append(1)

test_x = np.asarray(test_x)
for i in range(0, len(temp[0])-9):
    test_y.append(temp[9][i+9])
test_y = np.asarray(test_y)

print(len(test_y), len(test_x), len(test_x[0]))

567 567 487


先记录一下$y$值, 因为等下要进行缩放了

In [3]:
days = 12*20-24
hours = 24
y_data = []
for i in range(0, days*hours-9):
    y_data.append(data[9][i+9]) # 数据集里pm2.5正好在第10项
    
y_d = np.asarray(y_data)
print(len(y_d))

5175


x_d每行为一个$\vec x_i$

In [4]:
x_data = []
for i in range(0, days*hours-9): # 枚举起点
    x_data.append([])
    for j in range(0, 9):
        for k in range(0, 18):
            x_data[i].append(data[k][i+j])
            x_data[i].append(data[k][i+j]**2)
            x_data[i].append(data[k][i+j]**3)

    x_data[i].append(1) # for bias

x_d = np.asarray(x_data)
print(len(x_d), len(x_d[0]))

5175 487


Scaling

进行特征缩放, 对每个维度的值计算平均数$m_i$和标准差$\sigma _i$.

$x_i^r\gets\frac{x_i^r-m_i}{\sigma_i}$

用第$r$个例子中的第$i$个输入，减掉平均数$m_i$，然后除以标准差$\sigma_i$，得到的结果是所有的mean是0，所有的方差都是1.

$\sigma = \sqrt{\frac{1}{N}\sum{(x_i-\mu}))^2}$

In [5]:
mean = []
sigma = []

N = len(x_d)
for i in range(0, len(x_d[0])-1 ): # ignore bias
    summ = 0.0
    for j in range(0, N):
        summ = summ + x_d[j][i]
    m = summ/N
    mean.append(m)
    summ = 0.0
    for j in range(0, N):
        summ = summ + np.dot(x_d[j][i]-m, x_d[j][i]-m)/N

    sig = math.sqrt(summ)
    sigma.append(sig)
    for j in range(0, len(x_d)):
        x_d[j][i] = (x_d[j][i]-m)/sig

print(len(mean), len(sigma))

486 486


$\frac{\partial L}{\partial \vec w}=\sum_{i=1}^{n}- 2\vec x_i(y_i - \vec w\cdot\vec x_i)+2\lambda\vec w$

To update:

$\vec w^{i+1} = w^i - \eta\frac{\partial L}{\partial \vec w}$

使用Adagrad算法:

$\vec w^{t+1}\gets w^t -\frac{\eta^t}{\sigma^t}g^t$

$g^t=\frac{\partial L}{\partial\vec w},\sigma^t$:之前参数的所有微分的均方根

In [6]:
#w = np.random.rand(len(x_d[0]))
w = np.zeros(len(x_d[0]))

In [ ]:
eta = 0.001
lamb = 10
sum_w = 0.0
iteration = 1000000000

for i in range(iteration):
    loss = 0.0
    par_w = w
    for j in range(0, len(x_d)):
        loss = loss + ( y_d[j] - np.dot(w, x_d[j]) )**2
        par_w = par_w - ( y_d[j] - np.dot(w, x_d[j]))*(x_d[j])
#    point = random.randint(0, len(x_d)-1)
#    loss = loss + ( y_d[point] - np.dot(w, x_d[point]) )**2
#    par_w = par_w - ( y_d[point] - np.dot(w, x_d[point]))*(x_d[point])

    loss = loss + lamb*np.dot(w, w)
    par_w = par_w + lamb*w
    
    sum_w = sum_w + np.dot(par_w, par_w)
    
    sig = math.sqrt( sum_w/(i+1) )
    
    w = w - eta/sig*par_w
    #w = w - eta*par_w
    if i%100 == 0:
        print(sum_w, np.dot(par_w, par_w), loss)

41890.72651125808 41890.72651125808 161660.33145876564
4369620.012408477 43696.25110740408 161660.07665967612
8696260.831801271 43248.0538441134 161659.81955454368
13034605.508872692 43508.810399679045 161659.56714157376
17362810.412687816 43017.994746564786 161659.31590933882
21690064.840506382 42947.946831040004 161659.0686046608
26020459.62616803 43013.643761938605 161658.82451117525
30351888.622546144 43245.8603929346 161658.58366372882
34678118.45337985 43573.73127877291 161658.34585613114
38997981.79286311 43498.97997664044 161658.10986631995
43328068.77120593 43020.20175029013 161657.87565409398
47668047.66568027 43222.199090527974 161657.6457627928
51987542.15200145 43577.121187002514 161657.41890180897
56314770.408011556 43029.652257090595 161657.19252611688
60654942.69685892 43389.02753599295 161656.97090403843
64968772.16873248 43350.04301353751 161656.75089726865
69311554.28965214 43114.36589995655 161656.53295177617
73629924.74795759 43523.60654979441 161656.31903654084
77

645060132.0715877 43268.74886549527 161641.35147565632
649379038.4162526 43133.47870466517 161641.29768042665
653693232.4522817 43183.058541253966 161641.2448328556
658020108.3394653 43357.34512506111 161641.1927838944
662362005.4195474 43450.53585458847 161641.1410366615
666703596.5670754 43353.99264534922 161641.08932637455
671030066.0242069 43182.52466241329 161641.03792200284
675344299.5834892 43134.59854803164 161640.9872923952
679663180.1032506 43263.49639504173 161640.937559711
683998137.7843385 43421.02788682317 161640.88836497223
688342209.5051959 43427.601231183544 161640.8392742271
692678128.3808461 43277.31253362319 161640.79026843535
696998023.7601936 43141.03262515944 161640.74175086035
701312026.5888207 43169.97557757827 161640.69407787803
705636688.0306007 43330.57517296895 161640.6471645719
709976660.5179648 43444.65034967089 161640.60058064613
714319666.0700513 43386.747070966645 161640.55402178076
718650200.553563 43222.280868213114 161640.5076396389
722966543.766310

为mean和sigma加上bias对应的一项

In [21]:
print(len(test_x[0]), len(mean), len(sigma))
mean.append(0)
sigma.append(1.0)

487 486 486


将validation scaling一下

In [22]:
for i in range(0, len(test_x[0])):
    for j in range(0, len(test_x)):
        test_x[j][i] = (test_x[j][i]-mean[i])/sigma[i]

看看validation的准度

In [23]:
for i in range(0, len(test_x)):
    y = np.dot(test_x[i], w)
    print(y, test_y[i])

20.709036571134746 30.0
32.5213260347158 41.0
48.825945314076236 44.0
44.96303897866574 33.0
26.143377410728128 37.0
34.65888353212819 36.0
41.64093553366435 45.0
45.10385221482997 42.0
48.52121829825808 49.0
50.23186954690385 45.0
42.37509476574873 44.0
36.99018722446269 41.0
36.539353545381225 30.0
26.84180287394598 24.0
17.716085762835537 13.0
15.954587327591888 39.0
35.99859732785789 35.0
37.28970638111582 28.0
14.406907750946687 20.0
19.314965241908737 30.0
34.56145562098099 44.0
49.27530575804417 37.0
36.67776441360449 45.0
38.394256567611265 49.0
56.65511426447636 44.0
35.74103936624152 30.0
25.893754041342383 13.0
18.984114611003058 23.0
19.39135984321353 30.0
31.670000998701966 18.0
9.68352749753477 13.0
6.969201967575971 22.0
40.02044265833156 34.0
39.63006571009805 50.0
47.37837740044999 43.0
42.38674375050114 45.0
40.32789394281484 32.0
34.603319292150545 19.0
13.803343942201625 26.0
25.91043990126265 35.0
38.93678326670195 20.0
12.64206364570489 44.0
44.304179818668366 45.

In [24]:
submit_data_path = '/data/zys/regression/test.csv'
f = open(submit_data_path, 'r')


test_content = []

i = 0
for line in f.readlines():
    test_content.append([])
    j = 0
    for word in re.split(',|\n', line):
        if j>1:
            if word=='':
                continue
            if word=='NR':
                test_content[i].append(0.0)
            else:
                test_content[i].append(float(word))
        j = j+1
    i = i+1
f.close
print(len(test_content)/18)

test_data = []
idx = 0
for i in range(0, len(test_content), 18):
    test_data.append([])
    for j in range(0, 9):
        for k in range(0, 18):
            test_data[idx].append(test_content[i+k][j])
            test_data[idx].append(test_content[i+k][j]**2)
            test_data[idx].append(test_content[i+k][j]**3)
    test_data[idx].append(1) # for bias
    idx = idx + 1
    
for i in range(0, len(test_data[0])):
    for j in range(0, len(test_data)):
        test_data[j][i] = (test_data[j][i]-mean[i])/sigma[i]
    

240.0


In [25]:
for i in range(0, len(test_data)):
    y = np.dot(test_data[i], w)
    print("id_%d,%f"% (i, y))

id_0,8.865923
id_1,17.951893
id_2,24.922740
id_3,11.550816
id_4,27.819912
id_5,21.808357
id_6,21.719173
id_7,29.795875
id_8,16.010497
id_9,60.159712
id_10,11.340270
id_11,10.722575
id_12,63.109397
id_13,52.598076
id_14,22.169817
id_15,12.438864
id_16,31.809536
id_17,69.858446
id_18,-0.265546
id_19,15.689577
id_20,42.506627
id_21,72.403158
id_22,10.169795
id_23,17.563924
id_24,14.843119
id_25,39.174403
id_26,15.005490
id_27,61.046006
id_28,6.748921
id_29,54.576667
id_30,31.446414
id_31,11.313586
id_32,6.264622
id_33,19.227351
id_34,28.598519
id_35,37.366726
id_36,43.703654
id_37,29.949799
id_38,41.026356
id_39,35.475216
id_40,10.047675
id_41,40.222013
id_42,32.314279
id_43,53.056049
id_44,17.212515
id_45,35.112467
id_46,25.178320
id_47,12.018750
id_48,26.051531
id_49,31.779993
id_50,22.606886
id_51,8.151447
id_52,19.997912
id_53,52.393143
id_54,14.404257
id_55,36.486228
id_56,35.061197
id_57,20.149609
id_58,56.660330
id_59,23.147536
id_60,15.851552
id_61,41.306424
id_62,12.463416
id_63,